# unidbg
　　参考：[]()   
　　　　　[Git地址](https://github.com/zhkl0228/unidbg)   
　　　　　[Unidbg文档更新(一)](https://zhuanlan.zhihu.com/p/407839659)   
　　　　　[Unidbg文档慢更(二)](https://zhuanlan.zhihu.com/p/425355837)  
　　　　　[Unidbg使用指南(一)](http://www.91fans.com.cn/post/unidbgone/?ivk_sa=1024320u)   
　　　　　[爬虫工程师的unidbg入门教程](https://zhuanlan.zhihu.com/p/111793677)   
　　　　　[大杀器Unidbg真正的威力](https://bbs.pediy.com/thread-269056-1.htm)   

## 一、unidbg引入
## 二、unidbg概述
### 2.1、简介
　　unidbg 是一个基于 unicorn 的逆向工具，可以黑盒调用安卓和iOS中的 so 文件。unidbg 是一个标准的 java maven项目。    
　　现在的大多数app的签名算法都放到了so文件中，要想破解其中的签名算法，必须能够破解so文件。C++的逆向远比Java的逆向要难得多了，所以好多时候是没法破解的。此时，还可以采用hook的方法，直接读取程序中算出来的签名，但这样，需要实际运行这个应用，需要模拟器或者真机，效率又不是很高（比如frida的rpc调用，就需要后台运行应用）。  
　　unidbg是一个很巧妙地解决方案，他不需要直接运行app，也无需逆向so文件，而是通过在app中找到对应的JNI接口，然后用unicorn引擎直接执行这个so文件，所以效率也比较高。更方便的是目前利用 unidbg+springboot做成了web服务。  
  
### 2.2、特色优点
+ 模拟JNI调用API，以便可以调用JNI_OnLoad。
+ 支持JavaVM，JNIEnv。
+ 模拟syscalls调用。
+ 支持ARM32和ARM64。
+ 基于HookZz实现的inline hook。
+ 基于xHook实现的import hook。
+ 支持 iOS fishhook 、 substrate 、 whale hook。
+ 支持简单的控制台调试器，gdb，IDA android调试器服务器，指令跟踪，内存读/写跟踪。
+ 支持iOS objc和Swift运行时。

### 2.3、unidbg实现了以下lib的函数
+ libc.so
+ libcrypto.so
+ libdl.so
+ liblog.so
+ libm.so
+ libssl.so
+ libstdcpp.so
+ libz.so

### 2.4、依赖
+ unicorn
+ dynarmic
+ HookZz
+ xHook
+ AndroidNativeEmu
+ usercorn
+ keystone
+ capstone
+ idaemu
+ jelf
+ whale
+ kaitai_struct
+ fishhook
+ runtime_class-dump
+ mman-win32+ 

## 三、unidbg安装使用
### 3.1、下载unidbg项目
　　下载地址：https://github.com/zhkl0228/unidbg
### 3.2、导入到IDEA中
　　参考：[逆向工具之unidbg（在pc端模拟执行so文件中的函数）](https://hestyle.blog.csdn.net/article/details/106265603)  
　　unidbg项目用Java编写的，并且上面下载的是一个标准的maven项目（需本地JDK maven支持）。  
　　安装完依赖的目录结构应该如下：  
  
+ .idea/                   //idea配置相关
+ .mvn/                    //maven配置
+ assets/                  //项目图片图标资源
+ prebuilt/                //unicron支持
+ src/main                 //emulator源码
+ src/test/java            //一些demo
+ src/test/resouces        //要模拟的文件
+ target/                  //编译目标路径
+ target/classes/android   //依赖的lib和vfs,作者这里提供了SDK 19与23的库

### 3.3、测试unidbg
　　项目的src/test/java/com/xxxx/frameworks/core/encrypt路径中有一个TTEncrypt测试用例，直接执行其中的main方法，执行测试。  
　　控制台打印相关调用信息，说明项目导入成功。  
### 3.4、调用本地目标so文件

```java
package cn.hestyle;

import com.github.unidbg.Module;
import com.github.unidbg.arm.ARMEmulator;
import com.github.unidbg.linux.android.AndroidARMEmulator;
import com.github.unidbg.linux.android.AndroidResolver;
import com.github.unidbg.linux.android.dvm.*;
import com.github.unidbg.memory.Memory;


import java.io.File;
import java.io.IOException;

/**
 * description: EncryptUtils调用so
 *
 * @author hestyle
 * @version 1.0
 * @className unidbg->EncryptUtilsJni
 * @date 2020-05-20 22:01
 **/
public class EncryptUtilsJni extends AbstractJni {
    // ARM模拟器
    private final ARMEmulator emulator;
    // vm
    private final VM vm;
    // 载入的模块
    private final Module module;

    private final DvmClass TTEncryptUtils;

    /**
     *
     * @param soFilePath   需要执行的so文件路径
     * @param classPath    需要执行的函数所在的Java类路径
     * @throws IOException
     */
    public EncryptUtilsJni(String soFilePath, String classPath) throws IOException {
        // 创建模拟器实例，可以自定义模拟器，参考QDReaderJni
        //emulator = AndroidEmulatorBuilder.for64Bit().setProcessName("hellojni_2.0.3").build();
        // 创建app进程，包名可任意写
        emulator = new AndroidARMEmulator("cn.hestyle");
        // 模拟器的内存操作接口
        Memory memory = emulator.getMemory();
        // 设置系统类库解析: 作者支持19和23两个sdk
        memory.setLibraryResolver(new AndroidResolver(23));
        // 重点：  实现ModuleListener接口后通过下面的代码，设置模块加载监听
        memory.addModuleListener(this);

        // 创建DalvikVM，利用apk本身，可以为null
        vm = ((AndroidARMEmulator) emulator).createDalvikVM(null);
        // 创建Android虚拟机
        // vm = emulator.createDalvikVM(new File("unidbg-android/src/test/resources/unidbg/hellojni_2.0.3.apk"));
        // vm.setDvmClassFactory(new ProxyClassFactory());

        // 设置是否打印Jni调用细节
        vm.setVerbose(true);
        vm.setJni(this);

        // （关键处1）加载so到虚拟内存，填写so的文件路径
        DalvikModule dm = vm.loadLibrary(new File(soFilePath), false);
        // 调用jni
        dm.callJNI_OnLoad(emulator);
        module = dm.getModule();
        
        // （关键处2）加载so文件中的哪个类，填写完整的类路径
        TTEncryptUtils = vm.resolveClass(classPath);
    }

    /**
     * 调用so文件中的指定函数
     * @param methodSign 传入你要执行的函数信息，需要完整的smali语法格式的函数签名
     * @param args       是即将调用的函数需要的参数
     * @return 函数调用结果
     */
    private String myJni(String methodSign, Object ...args) {
        // 使用jni调用传入的函数签名对应的方法（）
        Number ret = TTEncryptUtils.callStaticJniMethod(emulator, methodSign, args);
        // ret存放返回调用结果存放的地址，获得函数执行后返回值
        StringObject str = vm.getObject(ret.intValue() & 0xffffffffL);
        return str.getValue();
    }

    /**
     * 关闭模拟器
     * @throws IOException
     */
    private void destroy() throws IOException {
        emulator.close();
        System.out.println("emulator destroy...");
    }

    public static void main(String[] args) throws IOException {
        // 1、需要调用的so文件所在路径
        String soFilePath = "src/test/resources/myso/libencryptLib.so";
        // 2、需要调用函数所在的Java类完整路径，比如a/b/c/d等等，注意需要用/代替.
        String classPath = "com/.../EncryptUtils";
        // 3、需要调用函数的函数签名，我这里调用EncryptUtils中的getGameKey方法，由于此方法没有参数列表，所以不需要传入
        String methodSign = "getGameKey()Ljava/lang/String;";
        EncryptUtilsJni encryptUtilsJni = new EncryptUtilsJni(soFilePath, classPath);
        // 输出getGameKey方法调用结果
        System.err.println(encryptUtilsJni.myJni(methodSign));
        encryptUtilsJni.destroy();
    }
}
```





## 四、
## 五、总结
　　unidbg 他不需要运行app，也无需逆向 so 文件，通过在 app 中找到对应的 JNI 接口，然后用 unicorn 引擎直接调用 so 文件，所以效率相对要高不少。
